In [1]:
import numpy as np
import dask.array as da
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58039 Dashboard: http://127.0.0.1:58040/status,Cluster Workers: 8 Cores: 8 Memory: 8.50 GB


### GIL analysis

In [2]:
a = da.random.random(size=(10000, 1000), chunks=(1000, 1000))
q, r = da.linalg.qr(a)
a2 = q.dot(r)

out = a2.compute()

### Comparison

In [8]:
def simulation_dask(init, years):
    y = da.from_array(np.log(init), chunks=(1000000, 1))
    for i in range(years+1):
        n_errors = da.random.normal(0, 0.1, size=(1000000,1), chunks=(1000000, 1))
        y = (1 - 0.2) * (np.log(init) + 0.03 * i) + 0.2 * y + n_errors
    y = da.exp(y)
    return y

In [9]:
def simulation_numpy(init, years):
    y = np.full((1000000, 1), np.log(init))
    for i in range(years+1):
        n_errors = np.random.normal(0, 0.1, (1000000, 1))
        y = (1 - 0.2) * (np.log(init) + 0.03 * i) + 0.2 * y + n_errors
    y = np.exp(y)
    return y

In [10]:
%time simulation_numpy(80000, 40).mean()

Wall time: 3.03 s


264957.69206603663

In [12]:
%time simulation_dask(80000, 40).mean().compute()

Wall time: 6.12 s


265020.26305006945